<a href="https://colab.research.google.com/github/johannes-kk/am207/blob/master/exercises/17_bbvi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DAY 17: Black-box Variational Inference


### AM207: Advanced Scientific Computing

#### Instructor: Weiwei Pan

#### Due: October 29th, 11:59pm EST

**Names of Group Members**:

Matthieu Meeus (matthieu_meeus@g.harvard.edu) 
 Maggie Wang (maggiewang@college.harvard.edu)
Nari Johnson njohnson@college.harvard.edu
Will Seaton (wseaton@g.harvard.edu)
Johannes Kolberg (johanneskolberg@g.harvard.edu)

## Learning Goals:

1. Implement BBVI and BBVI with the reparametrization trick
2. Compare the variance of score-function and reparametrization gradient esetimators
3. Understand the potential drawbacks of BBVI with the mean-field assumption


### Load necessary libraries

In [ ]:
from autograd import numpy as np
from autograd import scipy as sp
from autograd import grad
from autograd.misc.optimizers import adam, sgd

# Add elementwise for vectorized inputs
from autograd import elementwise_grad as egrad

In [ ]:
def black_box_variational_inference(logprob, D, num_samples):
    
    """
    Implements http://arxiv.org/abs/1401.0118, and uses the
    local reparameterization trick from http://arxiv.org/abs/1506.02557
    code taken from:
    https://github.com/HIPS/autograd/blob/master/examples/black_box_svi.py
    """

    def unpack_params(params):
        # Variational dist is a diagonal Gaussian.
        mean, log_std = params[:D], params[D:]
        return np.array(mean), np.array(log_std)

    def gaussian_entropy(log_std):
        return 0.5 * D * (1.0 + np.log(2*np.pi)) + np.sum(log_std)

    rs = npr.RandomState(0)
    def variational_objective(params, t):
        """Provides a stochastic estimate of the variational lower bound."""
        mean, log_std = unpack_params(params)
        samples = rs.randn(num_samples, D) * np.exp(log_std) + mean
        lower_bound = gaussian_entropy(log_std) + np.mean(logprob(samples, t))
        return -lower_bound

    gradient = grad(variational_objective)

    return variational_objective, gradient, unpack_params

def variational_inference(Sigma_W, sigma_y, y_train, x_train, forward, S, max_iteration, step_size, verbose):
    '''implements wrapper for variational inference via bbb for bayesian regression'''
    D = Sigma_W.shape[0]
    Sigma_W_inv = np.linalg.inv(Sigma_W)
    Sigma_W_det = np.linalg.det(Sigma_W)
    variational_dim = D
    
    #define the log prior on the model parameters
    def log_prior(W):
        constant_W = -0.5 * (D * np.log(2 * np.pi) + np.log(Sigma_W_det))
        exponential_W = -0.5 * np.diag(np.dot(np.dot(W, Sigma_W_inv), W.T))
        log_p_W = constant_W + exponential_W
        return log_p_W

    #define the log likelihood
    def log_lklhd(W):
        # S = W.shape[0]
        constant = (-np.log(sigma_y) - 0.5 * np.log(2 * np.pi)) * N
        exponential = -0.5 * sigma_y**-2 * np.sum((y_train.reshape((1, 1, N)) - forward(W, x_train))**2, axis=2).flatten()
        return constant + exponential

    #define the log joint density
    log_density = lambda w, t: log_lklhd(w) + log_prior(w)

    #build variational objective.
    objective, gradient, unpack_params = black_box_variational_inference(log_density, D, num_samples=S)

    def callback(params, t, g):
        if verbose:
            if  t % 100 == 0:
                print("Iteration {} lower bound {}; gradient mag: {}".format(t, -objective(params, t), np.linalg.norm(gradient(params, t))))

    print("Optimizing variational parameters...")
    #initialize variational parameters
    init_mean = np.ones(D)
    init_log_std = -100 * np.ones(D)
    init_var_params = np.concatenate([init_mean, init_log_std])
    
    #perform gradient descent using adam (a type of gradient-based optimizer)
    variational_params = adam(gradient, init_var_params, step_size=step_size, num_iters=max_iteration, callback=callback)
    
    return variational_params 

## Problem 1: Implementing BBVI
In this problem we implement BBVI and BBVI with the reparametrization trick.

For the following Bayesian model

\begin{align}
\mathbf{W}&\sim p(\mathbf{W})\\
Y^{(n)} &\sim p(Y^{(n)} | \mathbf{X}^{(n)}, \mathbf{W}^s),
\end{align}

the ***Black-box Variational Inference (BBVI)*** with the mean-field assumption algorithm approximates the posterior $p(\mathbf{W} | \mathrm{Data})$ with a mean-field (i.e. diagonal) Gaussian $q(\mathbf{W}) = \mathcal{N}(\mathbf{W}; \mu, \Sigma)$ by minimizing the KL-divergence between the approxaimte posterior and the true posterior:
0. **Initialization:** pick an intial value $\mu^{(0)}, \Sigma^{(0)}$
1. **Gradient Ascent:** repeat:

   1. Approximate the gradient 
   \begin{align}
   \nabla_{\mu, \Sigma} \, ELBO(\mathbf{W}) &= \mathbb{E}_{\mathbf{W} \sim q(\mathbf{W} | \mu, \Sigma)}\left[ \nabla_{\mu, \Sigma}\, q(\mathbf{W} | \mu, \Sigma) * \log \left( \frac{p(\mathbf{W}) \prod_{n=1}^N p(Y^{(n)} | \mathbf{X}^{(n)}, \mathbf{W})}{q(\mathbf{W} | \mu, \Sigma)} \right) \right]\\
   &\approx\frac{1}{S}\underbrace{\sum_{s=1}^S \nabla_{\mu, \Sigma}\, \log q(\mathbf{W}^s | \mu, \Sigma) * \log \left( \frac{p(\mathbf{W}^s) \prod_{n=1}^N p(Y^{(n)} | \mathbf{X}^{(n)}, \mathbf{W}^s)}{q(\mathbf{W}^s | \mu, \Sigma)} \right)}_{{\text{score function gradient}}},
   \end{align}
   where $\mathbf{W}^s\sim q(\mathbf{W} | \mu^{\text{current}}, \Sigma^{\text{current}})$.
   2. Update parameters $(\mu^{\text{current}}, \Sigma^{\text{current}}) \leftarrow (\mu^{\text{current}}, \Sigma^{\text{current}}) + \eta * {\text{score function gradient}}$

We've noted in lecture that this gradient estimate can have high variance and thus proposed a modified version of BBVI: The ***Black-box Variational Inference (BBVI) with the reparametrization trick***. 

BBVI with the reparametrization trick is as follows:
0. **Initialization:** pick an intial value $\mu^{(0)}, \Sigma^{(0)}$
1. **Gradient Ascent:** repeat:

   1. Approximate the gradient 
   \begin{align}
   \nabla_{\mu, \Sigma} \, ELBO(\mathbf{W}) \approx& \frac{1}{S} \sum_{s=1}^S \nabla_{\mu, \Sigma} \log \left[p(\epsilon_s^\top \Sigma^{1/2} + \mu) \prod_{n=1}^N p(Y^{(n)} | \mathbf{X}^{(n)}, \epsilon_s^\top \Sigma^{1/2} + \mu)\right] \\
   &- \nabla_{\mu, \Sigma}\underbrace{\mathbb{E}_{\mathbf{W} \sim \mathcal{N}(\mu, \Sigma )}\left[\log \mathcal{N}(\mathbf{W};\mu, \Sigma ) \right]}_{\text{Guassian entropy: has closed form}},
   \end{align}
   where $\epsilon_s \sim \mathcal{N}(0, \mathbf{I})$.
   2. Update parameters $(\mu^{\text{current}}, \Sigma^{\text{current}}) \leftarrow (\mu^{\text{current}}, \Sigma^{\text{current}}) + \eta * {\text{reparametrization gradient}}$

In [ ]:
mu_init = 1.
sigma_init = 1.
num_iteration = 10

mu_curr = mu_init
sigma_curr = sigma_init
S = 1

num_iterations = 50

gaussian_log_pdf = lambda mu, sigma_sq, W: -0.5 * (np.log(2 * np.pi * sigma_sq) + (x - mu)**2 / sigma_sq) 

def gaussian_pdf(mean, sd, x):
    var = float(sd)**2
    denom = (2*math.pi*var)**.5
    num = math.exp(-(float(x)-float(mean))**2/(2*var))
    return num/denom

def log_posterior_num():
  return np.log(0.5 * gaussian_pdf(-1., -0.5, W) + 0.8 * gaussian_pdf(4., 2., W))

def score_function_grad(mu_q, sigma_q):
  log_q = gaussian_log_pdf(mu_q, sigma_q, W)
  second_term = log_posterior_num(W) - log_q
  grad_obj = log_q * second_term
  return grad_obj

for i in range(num_iterations):
  # Sample S ws from q(W)
  W = np.random.normal(loc=mu_curr, scale=sigma_curr, size=S)

  mu_grad = grad(score_function_grad)(mu_q)
  sigma_grad = grad(score_function_grad)(sigma_q)

  mu_new = mu_curr + eta * np.mean(mu_grad)
  sigma_new = sigma_curr + eta * np.mean(sigma_grad)

  np.mean(sigma_grad)



NameError: ignored

In [ ]:
# NEW VERSION Johannes

mu_init = 1.
sigma_init = 2.

# Learning rate
eta = 1.0
# Dimension of weights (wouldn't this just be number of predictorws?)
num_weights = 1
# Samples per iteration (right?)
S = 3
# Iterations of gradient descent
num_iterations = 50

gaussian_log_pdf = lambda mu, sigma_sq, W: -0.5 * (np.log(2 * np.pi * sigma_sq) + (W - mu)**2 / sigma_sq) 

def gaussian_pdf(mean, sd, w):
    var = float(sd)**2
    denom = (2*np.pi*var)**.5
    num = np.exp(-(w-float(mean))**2/(2*var))
    return num/denom

def log_posterior_num(W):
  return np.log(0.5 * gaussian_pdf(-1., -0.5, W) + 0.8 * gaussian_pdf(4., 2., W))

def score_function_grad(mu_q, sigma_q, W):
  log_q = gaussian_log_pdf(mu_q, sigma_q, W)
  second_term = log_posterior_num(W) - log_q
  grad_obj = log_q * second_term
  return grad_obj

np.random.seed(42)

elbo_grad_mu = 0
elbo_grad_sigma = 0

elbo_grad = 0

mu_new = mu_init
sigma_new = sigma_init
for i in range(num_iterations):
  print('Iteration', i)
  for s in range(S):
    #print('   Sample', s)
    # Sample S ws from q(W)
    W = np.random.normal(loc=mu_new, scale=sigma_new, size=num_weights)

    elbo_grad += egrad(score_function_grad)(mu_new, sigma_new, W)
    #elbo_grad_mu += egrad(score_function_grad)(mu_new)
    #elbo_grad_sigma += egrad(score_function_grad)(sigma_new)

  elbo_grad /= num_iterations
  #elbo_grad_mu /= num_iterations
  #elbo_grad_sigma /= num_iterations

  mu_new += eta*elbo_grad
  sigma_new += eta*elbo_grad
  print(' ', mu_new, sigma_new)
  #mu_new = mu_curr + eta * np.mean(mu_grad)
  #sigma_new = sigma_curr + eta * np.mean(sigma_grad)


Iteration 0
  1.0236031897525026 2.0236031897525026
Iteration 1
  1.194023138803256 2.194023138803256
Iteration 2
  1.4215570719121544 2.4215570719121544
Iteration 3
  1.4396607713347005 2.4396607713347005
Iteration 4
  1.1986010461720564 2.1986010461720564
Iteration 5
  1.1066373738593434 2.1066373738593436
Iteration 6
  1.0746461488360513 2.0746461488360515
Iteration 7
  0.9527159150496989 1.952715915049699
Iteration 8
  0.8485054248310347 1.848505424831035
Iteration 9
  0.8358869563690503 1.8358869563690505
Iteration 10
  1.0766400772405142 2.0766400772405142
Iteration 11
  0.9238948828176237 1.9238948828176237
Iteration 12
  0.6969543379323011 1.696954337932301
Iteration 13
  0.7023109002928137 1.7023109002928136
Iteration 14
  0.5929428337153924 1.5929428337153924
Iteration 15
  0.5997460602779957 1.5997460602779956
Iteration 16
  0.4735198181404672 1.473519818140467
Iteration 17
  0.4497944686995407 1.4497944686995405
Iteration 18
  0.45963489832567844 1.4596348983256784
Iteratio


**Exercise 1:** Let's set the unnormalized true posterior distribution to be:
\begin{align}
p(\mathbf{W}) \prod_{n=1}^N p(Y^{(n)} | \mathbf{X}^{(n)}, \mathbf{W}) = 0.5 \mathcal{N}(\mathbf{W}; -1, 0.5) + 0.8 \mathcal{N}(\mathbf{W}; 4, 2)
\end{align}
where $\mathbf{W} \in \mathbb{R}$.

Implement the score-function gradient estimator as well as the reparametrization gradient estimator. Which estimator has higher variance? Try $S = 1, 10, 30, 50, 100$.

**Exercise 2:** Implement BBVI with the reparametrization trick (see sample code in notes for Lecture #17, you'll need to modify the `variational_inference` function and pass the above unnormalized posterior when we call `black_box_variational_inference`, instead of passing it the `logdensity` that is constructed in the current version of `variational_inference`). Apply it to approximate the above unnormalized posterior distribution. Plot both the unnormalized posterior and the optimal approximate posterior. How well does your BBVI solution capture the true unnormalized posterior? Is this expected?

You might want to change the form of the unnormalized posterior (try adding more mixture components to the mixture model) and see how well our BBVI does. 

Based on these experiments, what are the potential draw backs of our realization of BBVI? What can we do to improve our posterior approximation?

In [ ]:
def generate_data(N, y_var=1.):
    '''generate training data with a gap, testing data uniformly sampled'''


    # Sample from the posterior p(w | x, y)
    # Use this sample of w and our forward to compute y from x_train

    #training x
    x_train = np.hstack((np.linspace(-10, -5, N), np.linspace(5, 10, N)))
    #function relating x and y
#     f = lambda x:  0.01 * x**3

    f = lambda x: 2 * x
    #y is equal to f(x) plus gaussian noise
    y_train = f(x_train) + np.random.normal(0, y_var**0.5, 2 * N)

    ## generate testing data
    #nubmer of testing points
    N_test = 100
    #testing x
    x_test = np.linspace(-10, 10, N_test)
    y_test = f(x_test) + np.random.normal(0, y_var**0.5, N_test)

    return x_train, y_train, x_test, y_test



In [ ]:
def bayesian_polynomial_regression(x, y, x_test, y_test, prior_var, y_var, ax, S=100, poly_degree=10):
    '''visualize the posterior predictive of a Bayesian polynomial regression model'''
    poly = PolynomialFeatures(poly_degree)
    #transform training x: add polynomial features
    x_poly = poly.fit_transform(x.reshape((-1, 1)))
    #transform x_test: add polynomial features
    x_test_poly = poly.fit_transform(x_test.reshape((-1, 1)))
    #Gaussian log pdf
    gaussian_log_pdf = lambda mu, sigma_sq, x: -0.5 * (np.log(2 * np.pi * sigma_sq) + (x - mu)**2 / sigma_sq)

    #reshape y into 2D array
    y_matrix = y.reshape((-1, 1))

    #define the covariance and precision matrices of the prior on the weights
    prior_variance = np.diag(prior_var * np.ones((x_poly.shape[1], )))
    prior_precision = np.linalg.inv(prior_variance)

    #defining the posterior variance
    joint_variance = np.linalg.inv(prior_precision + 1. / y_var * x_poly.T.dot(x_poly))
    #defining the posterior mean
    joint_mean = joint_variance.dot(x_poly.T.dot(y_matrix)) * 1. / y_var

    #sampling S points from the posterior
    posterior_samples = np.random.multivariate_normal(joint_mean.flatten(), joint_variance, size=S)
    #sampling S points from the posterior predictive
    y_predict_noiseless = np.array([x_test_poly.dot(sample) for sample in posterior_samples])
    y_predict_bayes = y_predict_noiseless + np.random.normal(0, y_var**0.5, size=(S, len(x_test)))
    
    #compute log likelihood for the test data
    log_likelihood_bayes = []
    for n in range(len(y_test)):
        log_likelihood_bayes.append(gaussian_log_pdf(y_predict_noiseless[:, n], y_var, y_test[n]).mean())
    log_likelihood_bayes = np.array(log_likelihood_bayes)
    
    #compute the 95 percentiles of the posterior predictives
    ub_bayes = np.percentile(y_predict_bayes, 97.5, axis=0)
    lb_bayes = np.percentile(y_predict_bayes, 2.5, axis=0)
    
    #visualize the posterior predictive distribution
    ax.scatter(x, y, color='red', s=10, alpha=0.5, label='train data')
    ax.fill_between(x_test, ub_bayes, lb_bayes, color='blue', alpha=0.2, label="test log-likelihood: {}".format(np.round(np.sum(log_likelihood_bayes), 4)))
    ax.set_title('posterior predictive distribution of bayesian regression model with prior variance of {}'.format(prior_var))
    ax.legend(loc='best')
    ax.set_xlim([-10, 10])
    
    return ax, joint_variance, joint_mean

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
import autograd.numpy.random as npr



In [ ]:
##set random seed
rand_state = 0
random = np.random.RandomState(rand_state)

##generate training data
#number of points in each of the two segments of the domain
N = 20
#output variance
y_var = 1.
x_train, y_train, x_test, y_test = generate_data(N, y_var)

##transform covariates for polynomial regression
poly = PolynomialFeatures(1)
#transform x: add polynomial features
x_poly = poly.fit_transform(x_train.reshape((-1, 1)))
#transform x_test: add polynomial features
x_test_poly = poly.fit_transform(x_test.reshape((-1, 1)))

##define dimensions
N = x_poly.shape[0]
D = x_poly.shape[1]

##define variances
sigma_y = 1.**2
weight_noise = 5**2
Sigma_W = weight_noise * np.eye(D)

##polynomial function
def forward(w, x):
    x_poly = poly.fit_transform(x.reshape((-1, 1))) 
    return np.dot(w, x_poly.T)

In [ ]:
#S = 1
S_arr = [1, 10, 30, 50, 100]
max_iteration = 2000
step_size = 1e-2
verbose=False

In [ ]:
for S in S_arr:
  S_param_estimates = []
  for j in range(1):
    S_param_estimates.append(variational_inference(Sigma_W, sigma_y, y_train, x_train, forward, S, max_iteration, step_size, verbose))
  
  print(S_param_estimates)
  S_var = np.var(S_param_estimates)
  print(S_var)
  print('Variance for ' + str(S) + ' is ' + str(S_var))
  print('\n')

Optimizing variational parameters...
[array([-5.38335290e-03,  1.99173220e+00, -8.00000002e+01, -8.00000002e+01])]
1640.472142698992
Variance for 1 is 1640.472142698992


Optimizing variational parameters...
[array([-5.38335290e-03,  1.99173220e+00, -8.00000002e+01, -8.00000002e+01])]
1640.472142698992
Variance for 10 is 1640.472142698992


Optimizing variational parameters...
[array([-5.38335290e-03,  1.99173220e+00, -8.00000002e+01, -8.00000002e+01])]
1640.472142698992
Variance for 30 is 1640.472142698992


Optimizing variational parameters...
[array([-5.38335290e-03,  1.99173220e+00, -8.00000002e+01, -8.00000002e+01])]
1640.472142698992
Variance for 50 is 1640.472142698992


Optimizing variational parameters...


KeyboardInterrupt: ignored

In [ ]:
variational_params

array([ -0.1357079 ,   2.01506752, -80.0000002 , -80.0000002 ])